In [9]:
import requests
from bs4 import BeautifulSoup
import time
from defs import get_movie

In [2]:
with open('movies.csv') as movies:
    movies.readline()  #ignore the header
    movies_list = [movie.strip().split(',') for movie in movies.readlines()]
    
data = list(enumerate(movies_list))

In [3]:
movie_idx_from = int(input('Enter the movie index from where you want to scrape:')) - 1
movie_idx_to = int(input('Enter the movie index to where you want to scrape:'))

Enter the movie index from where you want to scrape:1
Enter the movie index to where you want to scrape:100


In [ ]:
start = time.time()
all_movies = []
idx = movie_idx_from + 1
for movie_id, lang_id in movies_list[movie_idx_from:movie_idx_to]:
    print(f"{idx}. movie_id: {movie_id} lang_id: {lang_id}")
    search_string = f"https://www.cbfcindia.gov.in/main/search-result?movie_id={movie_id}&lang_id={lang_id}"
    response = requests.get(search_string)
    soup = BeautifulSoup(response.content)

    movie_details = [each.text for each in soup.find_all('td')[1:]]
    keys = ['Movie No.'] + movie_details[::2]
    values = [idx] + movie_details[1::2]
    movie_dict = dict(zip(keys, values))    
    all_movies.append(movie_dict)
    idx += 1
end = time.time()
print(end- start)

In [12]:
# def get_movie(row,a,b):  # https://stackoverflow.com/questions/41385708/multiprocessing-example-giving-attributeerror
#     idx = row[0]
#     movie_id = row[1][0]
#     lang_id = row[1][1]
#     print(f"{idx}. movie_id: {movie_id} lang_id: {lang_id}")
#     search_string = f"https://www.cbfcindia.gov.in/main/search-result?movie_id={movie_id}&lang_id={lang_id}"
#     response = requests.get(search_string)
#     soup = BeautifulSoup(response.content)

#     movie_details = [each.text for each in soup.find_all('td')[1:]]
#     keys = ['Movie No.'] + movie_details[::2]
#     values = [idx] + movie_details[1::2]
#     movie_dict = dict(zip(keys, values))    
#     return movie_dict
#     idx += 1


In [10]:
import multiprocessing as mp
idx = movie_idx_from + 1
pool = mp.Pool(mp.cpu_count())

start = time.time()
results = [pool.apply(get_movie, args=(row, 4, 8)) for row in data[movie_idx_from:movie_idx_to]]
end = time.time()
print(end- start)

330.60239696502686


In [6]:
import multiprocessing as mp
mp.cpu_count()

16

In [ ]:
import pandas as pd
all_movies_df = pd.DataFrame(all_movies)
all_movies_df.head()
all_movies_df.to_csv(f'movies_{movie_idx_from + 1}_{movie_idx_to}.csv', index=False)